In [1]:
import os
import torch
import json
import argparse
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image

from models import get_image_encoder_llava, encode_image_llava, get_llava_model, get_response_llava
from models import get_image_encoder_internlm, encode_image_internlm, get_internlm_model, get_response_internlm
from models import encode_image_clip, get_image_encoder_clip, i2t_similarity_clip

from datasets import collate_fn, SingleTargetPairedImageDataset, PairedImageDataset, ImageDataset
from poison_utils import load_image, load_image_tensors, L2_norm, save_poison_data, test_attack_efficacy
from torch.nn import DataParallel

from torchvision import transforms

# diff augmentation
# import kornia
from augmentation_zoo import *


def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def embedding_attack_Linf(encode_images, image_base, image_target, emb_dist, masks, orig_sizes_base, orig_sizes_target, \
                     iters=100, lr=1/255, eps=8/255, diff_aug=None, resume_X_adv=None, target_caption=""):
      '''
      optimizing x_adv to minimize emb_dist( img_embed of x_adv, img_embed of image_target ) within Lp constraint using PGD

      image_encoder: the image embedding function (e.g. CLIP, EVA)
      image_base, image_target: images BEFORE normalization, between [0,1]
      emb_dist: the distance metrics for vision embedding (such as L2): take a batch of bs image pairs as input, \
            and output EACH of pair-wise distances of the whole batch (size = [bs])

      eps: for Lp constraint
      lr: the step size. The update is grad.sign * lr
      diff_aug: using differentiable augmentation, e.g. RandomResizeCrop
      resume_X_adv: None or an initialization for X_adv

      return: X_adv between [0,1]
      '''
      # assert len(image_base.size()) == len(image_target.size()) and len(image_base.size()) == 4, 'image size length should be 4'
      # assert image_base.size(0) == image_target.size(0), 'image_base and image_target contain different number of images'
      bs = image_base.size(0)
      device = image_base.device

      with torch.no_grad():
            embedding_targets = encode_images(image_target, bs, orig_sizes_target)

      X_adv = image_base.clone().detach() + (torch.rand(*image_base.shape)*2*eps-eps).to(device)
      # X_adv = image_base.clone().detach()
      if resume_X_adv is not None:
            print('Resuming from a given X_adv')
            X_adv = resume_X_adv.clone().detach()
      X_adv.data = X_adv.data.clamp(0,1)
      X_adv.requires_grad_(True) 

      optimizer = optim.SGD([X_adv], lr=lr)
      scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[int(iters*0.5)], gamma=0.5)

      loss_best = 1e8 * torch.ones(bs).to(device)
      X_adv_best = resume_X_adv.clone().detach() if resume_X_adv is not None else torch.rand(*image_base.shape).to(device)

      batch_losses = []

      # for i in tqdm(range(iters)):
      for i in range(iters):
            embs = encode_images(X_adv, bs, orig_sizes_base)
            

            loss = emb_dist(embs, embedding_targets) # length = bs

            # if i% max(int(iters/1000),1) == 0:
            #       if (loss < loss_best).sum()>0:
            #             index = torch.where(loss < loss_best)[0]
            #             loss_best[index] = loss.clone().detach()[index].to(loss_best[index].dtype)
            #             X_adv_best[index] = X_adv.clone().detach()[index]

            loss = loss.sum() 
            # print("loss shape", loss)
            sim_score = i2t_similarity_clip(image_encoder, image_processor, X_adv[0][:, :original_sizes_base[0][1], :original_sizes_base[0][0]], target_caption)
            # print("sim_score shape", sim_score.shape)
            loss -= sim_score[0][0]


            #### edit when returning to joint loss #####
            if i % max(int(iters / 1000), 1) == 0:
                  # Ensure `loss` is a tensor with per-batch losses
                  loss_per_batch = -sim_score  # Assuming `sim_score` is a tensor with shape [bs]

                  if (loss_per_batch < loss_best).sum() > 0:
                        index = torch.where(loss_per_batch < loss_best)[0]
                        loss_best[index] = loss_per_batch.clone().detach().to(loss_best[index].dtype)
                        X_adv_best[index] = X_adv.clone().detach()[index]
            #######

            optimizer.zero_grad()
            loss.backward()

            # print losses at increments
            # if i% max(int(iters/20),1) == 0:
            #       batch_losses.append(loss.item()/bs)
            #       print('Iter :{} loss:{:.4f}, lr * 255:{:.4f}'.format(i,loss.item()/bs, scheduler.get_last_lr()[0]*255))

            for j in range(bs):
                  X_adv.grad[j] *= (1 - masks[j])

            # Linf sign update
            X_adv.grad = torch.sign(X_adv.grad)
            optimizer.step()
            scheduler.step()
            X_adv.data = torch.minimum(torch.maximum(X_adv, image_base - eps), image_base + eps) #
            X_adv.data = X_adv.data.clamp(0,1)     
            X_adv.grad = None  

            if torch.isnan(loss):
                  print('Encounter nan loss at iteration {}'.format(i))
                  break                 

      with torch.no_grad():
            embs = encode_images(X_adv_best, bs, orig_sizes_base)
            loss = emb_dist(embs, embedding_targets)
            # print('Best Total loss vector:{}'.format(loss))
            print('Best Total loss:{:.4f}'.format(loss.mean().item()))

      return X_adv_best, loss.detach(), batch_losses




[2025-01-01 11:31:06,345] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
task = "mini_MathVista_grid"
args = {
    "model": "clip",
    "poison_save_pth": f"./data/poisons/{task}",
    "iter_attack": 500,
    "lr_attack": 0.2,
    "task_data_pth": f"data/task_data/{task}",
    "diff_aug_specify": None,
    "temperature": 0,
    "max_new_tokens": 200,
    "training_mode": "all_target",
    "eps": 16,
    "batch_size": 1,
}

image_encoder, image_processor, diff_aug, img_size = get_image_encoder_clip()
encode_images = lambda imgs, bs, orig_sizes: encode_image_clip(image_encoder, imgs, img_size, bs, diff_aug, orig_sizes) 
chat = None

question_json = read_json(os.path.join(args["task_data_pth"], "questions.json"))
pids = question_json.keys()

images_base, images_target, target_caps = load_image_tensors(args["task_data_pth"],img_size,args["training_mode"])


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading image tensors: 100%|██████████| 19/19 [00:00<00:00, 25.72it/s]

Finishing loading 100 pairs of base and target images for poisoning, size=100


In [3]:
base_dataset = ImageDataset(images_base)
target_dataset = ImageDataset(images_target)

import torch.nn.functional as F
import random
import torch


def collate_fn_image(batch):
    images_base = batch
    max_height = max(img.size(2) for img in images_base)
    max_width = max(img.size(3) for img in images_base)
    
    # Create lists to store padded images and masks
    padded_images_base = []
    original_sizes = []
    masks = []

    for img in images_base:
        _, __, h, w = img.shape  # Channels, Height, Width
        original_sizes.append((w, h))  # Store original size (width, height)

        # Calculate padding (left, right, top, bottom)
        padding = (0, max_width - w, 0, max_height - h)

        # Pad the image
        padded_img = F.pad(img, padding, value=0)  # Zero-padding
        padded_images_base.append(padded_img)

        # Create a padding mask (1 for padded, 0 for original)
        mask = torch.ones(1, max_height, max_width)  # Initialize mask with ones
        mask[:, :h, :w] = 0  # Set original area to 0 (no padding)
        masks.append(mask)
    
    images_base = torch.cat(padded_images_base, dim=0)
    masks = torch.cat(masks, dim=0)

    return images_base, masks, original_sizes

base_dataloader = torch.utils.data.DataLoader(
    base_dataset, batch_size=args["batch_size"], 
    shuffle=False, collate_fn=collate_fn_image
)

target_dataloader = torch.utils.data.DataLoader(
    target_dataset, batch_size=1, 
    shuffle=False, collate_fn=collate_fn_image
)

In [4]:
# next(iter(target_dataloader))

In [5]:
################################################################################

###########    implement grid approach here. use dataloaders above   ###########

################################################################################

import matplotlib.pyplot as plt





###### Running attack optimization ######
original_sizes_list = []
X_adv_list = []
all_losses = []
loss_attack_list = []
start_idx = 0
end_idx = 0
for j, (image_target, _, original_sizes_target) in  enumerate(target_dataloader):
    target_name = target_caps[j]["name"]
    target_path = os.path.join(args["poison_save_pth"], target_caps[j]["name"])
    if not os.path.exists(target_path):
        os.makedirs(target_path)

    ###### Resume by checking already saved images ######
    saved_images = set(os.listdir(target_path))
    saved_image_ids = {int(fname.split('.')[0]) for fname in saved_images if fname.endswith('.jpg')}

    ###### show target images ######
    # print("Working on target:", target_name)
    # plt.imshow(image_target.squeeze(0).permute(1,2,0))
    # plt.axis("off")
    # plt.show()

    #### sanity check ####
    with torch.no_grad():
        sim_score = i2t_similarity_clip(image_encoder, image_processor, image_target, target_caps[j]["caption"]).item()
        print("Caption:", target_caps[j]["caption"])
        print(f"Caption similarity score target: {sim_score:.4f}")
    ####

    # print(image_target.squeeze(0).permute(1,2,0).shape)
    # for i, (image_base, masks_base, original_sizes_base) in  enumerate(base_dataloader):
    for i, (image_base, masks_base, original_sizes_base) in  tqdm(enumerate(base_dataloader), total=len(base_dataloader)):
        # if i == 1:
        #       raise Exception("Stopping on purpose here")
        # plt.imshow(image_base.squeeze(0).permute(1,2,0))
        # plt.axis("off")
        # plt.show()

        start_idx = end_idx
        end_idx = start_idx + image_base.size(0) # mathvista

        if all(idx%100 in saved_image_ids for idx in range(start_idx, end_idx)):
                print(f'Batch {i} already processed, skipping...')
                continue

        print('batch_id = ',i)
        image_base, image_target, masks_base = image_base.cuda(), image_target.cuda(), masks_base.cuda()


        #### sanity check start ####
        with torch.no_grad():
            sim_score = i2t_similarity_clip(image_encoder, image_processor, image_base, target_caps[j]["caption"]).item()
            print(f"Caption similarity score base: {sim_score:.4f}")
        ####

        X_adv, loss_attack, batch_losses = embedding_attack_Linf(
                encode_images=encode_images, image_base=image_base, 
                image_target=image_target, emb_dist=L2_norm, 
                iters=args["iter_attack"], lr=args["lr_attack"]/255, 
                eps=int(args["eps"])/255, diff_aug=diff_aug, resume_X_adv=None, 
                masks=masks_base, orig_sizes_base=original_sizes_base, orig_sizes_target=original_sizes_target,
                target_caption=target_caps[j]["caption"]
        )


        if chat:
                if "MathVista" not in args["task_data_pth"]:
                    print()
                    print("Before:", chat(image_base[0][:][:original_sizes_base[0][1]][:original_sizes_base[0][0]]))
                    print("After:", chat(X_adv[0][:][:original_sizes_base[0][1]][:original_sizes_base[0][0]]))
                    print()
                else:
                    print("Question:", question_json[f"{i+1}"]["question"])
                    print()
                    print("Actual Answer:", question_json[f"{i+1}"]["answer"])
                    print("-"*20)
                    print("Model Answer Before:", chat(image_base[0][:][:original_sizes_base[0][1]][:original_sizes_base[0][0]], i+1))
                    print("-"*20)
                    print("Model Answer After: ", chat(X_adv[0][:][:original_sizes_base[0][1]][:original_sizes_base[0][0]], i+1))
                    print()
        
        #### sanity check start ####
        sim_score = i2t_similarity_clip(image_encoder, image_processor, X_adv, target_caps[j]["caption"]).item()
        print(f"Caption similarity score adv: {sim_score:.4f}")
        ####


        # ###### Save poisoned images after each batch ######
        for k in range(X_adv.size(0)):  # Save each image in the batch
                img_idx = start_idx + k
                img_pth = os.path.join(target_path, f'{img_idx%100}.jpg')
                if img_idx not in saved_image_ids:  # Only save if it doesn't already exist
                    img_tensor = X_adv[k][:, :original_sizes_base[k][1], :original_sizes_base[k][0]]
                    print("Max Pixel Difference between Adversarial Image and Base *255:", round(torch.max(torch.abs(X_adv-image_base)).item()*255, 4))
                    save_image(img_tensor.cpu(), img_pth)
                    print(f'Saved poisoned image {img_idx} to {img_pth}') # mathvista
        
        ######## show noise ##############
        amplification_factor = 30  # Increase the factor to make the noise more visible
        noise = (X_adv - image_base) * amplification_factor

        # Normalize the noise to [0, 1] for visualization
        noise = (noise - noise.min()) / (noise.max() - noise.min())

        # Display the noise
        # plt.imshow(noise.squeeze(0).permute(1, 2, 0).cpu().numpy())  # Ensure the tensor is on CPU and converted to numpy
        # plt.axis("off")
        # plt.title("Adversarial Noise (Amplified)")
        # plt.show()
        #######################


        # X_adv_list.append(X_adv)

        X_adv_list += X_adv 
        # print(torch.tensor(original_sizes).size())
        original_sizes_list.append(torch.tensor(original_sizes_base))
        loss_attack_list.append(loss_attack)
        all_losses.append(batch_losses)

        # save loss tensor here

# X_adv = torch.cat(X_adv_list,axis=0).cpu()
loss_tensor_path = f'./results/loss_tensor_{args["task_data_pth"].split("/")[-1]}.pt'

X_adv = X_adv_list
original_sizes_base = torch.cat(original_sizes_list,axis=0)
loss_attack = torch.cat(loss_attack_list,dim=0)
torch.save(torch.tensor(batch_losses), loss_tensor_path)

Caption: a cartoon illustration of a living room with a chair and table
Caption similarity score target: 27.6583


 34%|███▍      | 34/100 [00:00<00:00, 337.42it/s]

Batch 0 already processed, skipping...
Batch 1 already processed, skipping...
Batch 2 already processed, skipping...
Batch 3 already processed, skipping...
Batch 4 already processed, skipping...
Batch 5 already processed, skipping...
Batch 6 already processed, skipping...
Batch 7 already processed, skipping...
Batch 8 already processed, skipping...
Batch 9 already processed, skipping...
Batch 10 already processed, skipping...
Batch 11 already processed, skipping...
Batch 12 already processed, skipping...
Batch 13 already processed, skipping...
Batch 14 already processed, skipping...
Batch 15 already processed, skipping...
Batch 16 already processed, skipping...
Batch 17 already processed, skipping...
Batch 18 already processed, skipping...
Batch 19 already processed, skipping...
Batch 20 already processed, skipping...
Batch 21 already processed, skipping...
Batch 22 already processed, skipping...
Batch 23 already processed, skipping...
Batch 24 already processed, skipping...
Batch 25 a

100%|██████████| 100/100 [00:00<00:00, 232.89it/s]


Batch 40 already processed, skipping...
Batch 41 already processed, skipping...
Batch 42 already processed, skipping...
Batch 43 already processed, skipping...
Batch 44 already processed, skipping...
Batch 45 already processed, skipping...
Batch 46 already processed, skipping...
Batch 47 already processed, skipping...
Batch 48 already processed, skipping...
Batch 49 already processed, skipping...
Batch 50 already processed, skipping...
Batch 51 already processed, skipping...
Batch 52 already processed, skipping...
Batch 53 already processed, skipping...
Batch 54 already processed, skipping...
Batch 55 already processed, skipping...
Batch 56 already processed, skipping...
Batch 57 already processed, skipping...
Batch 58 already processed, skipping...
Batch 59 already processed, skipping...
Batch 60 already processed, skipping...
Batch 61 already processed, skipping...
Batch 62 already processed, skipping...
Batch 63 already processed, skipping...
Batch 64 already processed, skipping...


...

Best Total loss:14.1172
Caption similarity score adv: 72.5493
Max Pixel Difference between Adversarial Image and Base *255: 16.0
Saved poisoned image 1342 to ./data/poisons/mini_MathVista_grid/sci/42.jpg
batch_id =  43
Caption similarity score base: 11.4546


 44%|████▍     | 44/100 [46:19<1:07:59, 72.85s/it]

Best Total loss:13.8438
Caption similarity score adv: 75.3770
Max Pixel Difference between Adversarial Image and Base *255: 16.0
Saved poisoned image 1343 to ./data/poisons/mini_MathVista_grid/sci/43.jpg
batch_id =  44
Caption similarity score base: 3.4930


 45%|████▌     | 45/100 [47:31<1:06:34, 72.63s/it]

Best Total loss:14.3906
Caption similarity score adv: 74.5592
Max Pixel Difference between Adversarial Image and Base *255: 16.0
Saved poisoned image 1344 to ./data/poisons/mini_MathVista_grid/sci/44.jpg
batch_id =  45
Caption similarity score base: 7.6292
